In [1]:
from IPython.core.display import HTML

In [2]:
from langchain_community.llms import Ollama

# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

/tmp/ipykernel_297160/4093896791.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")


In [3]:
llm.invoke('hi')

'Hello! How can I assist you today? If you have any questions or need information, feel free to ask. 😊'

In [4]:
llm.invoke('سلام. خوبی؟')

'سلام! من عالی هستم، تشکر که سوال پرسیدید. اگر چیزی داشته باشید به کمک باید یا سوالی دارید، حتماً مطمئن شوید درباره آن بپرسید!'

In [5]:
from langchain.chains import LLMMathChain

llm_math = LLMMathChain.from_llm(llm, verbose=True)

In [6]:
llm_math.prompt.template

'Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'

In [1]:
# llm_math.run("what is 13 raised to the .3432 power?")

In [3]:
# i = 0
# example_query = "What is 551368 divided by 82"
# llm_math.invoke(example_query)

In [3]:
from langchain.chains import LLMMathChain
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate

# Create a custom prompt template to enforce the expected output.
custom_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a expert math solver. Solve the problem below and output only the final answer in the exact format:\n"
        "Answer: <result>\n\n"
        "Do not include any explanation, commentary, or extra text.\n\n"
        "Round the result with 2. For example,if the result is 5.6666666, make the result 5.66.\n\n"
        "Problem: {question}"
    )
)


# custom_prompt = PromptTemplate(
#     input_variables=["question"],
#     template=(
#         "You are an expert math solver. Solve the following problem using exact arithmetic. "
#         "Only output the final answer in the exact format 'Answer: <result>' with no additional text.\n\n"
#         "Problem: {question}"
#     )
# )




# Use your local Ollama model.
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434", temperature = 0)

# Instantiate the math chain with the custom prompt.
math_chain = LLMMathChain(llm=llm, prompt=custom_prompt, verbose = True)

# Run the chain with the math problem.
result = math_chain.invoke("What is 551368 divided by 82?")
# result = math_chain.invoke("What is 551121 divided by 82?")
# result = math_chain.invoke("What is (12 plus 18 minus 10) devide by 3?")
print(result)  # Expected output: Answer: 6724


/home/mahdi/Math_LangChain/.venv/lib/python3.12/site-packages/langchain/chains/llm_math/base.py:175: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
What is 551368 divided by 82?Answer: 6724.39
> Finished chain.
{'question': 'What is 551368 divided by 82?', 'answer': 'Answer: 6724.39'}


In [9]:
print("LLMMathChain input keys:", llm_math.input_keys)


LLMMathChain input keys: ['question']


In [11]:
import numexpr as ne
computed_result = ne.evaluate("551368 / 82")
print("Computed numeric result:", computed_result)


Computed numeric result: 6724.0


## understand which formula is needed

<h3>Using Router is a common and effective pattern in LangChain for handling multi-step workflows.</h3> 

<p>Using a router chain that outputs a structured dictionary—typically with keys like "destination" (to select the appropriate chain) and "next_inputs" (to pass along additional context or parameters)—helps separate the decision logic from the execution logic.</p>



<h3>Here’s why this approach is beneficial:</h3>
<ul>
    <li><strong>Modularity: </strong> It decouples the routing logic from the individual chain implementations, allowing each chain to focus on its specific task.</li>
    <li><strong>Scalability: </strong> You can easily add or modify destination chains without changing the overall structure.</li>
    <li><strong>Clarity: </strong> The structured output makes it explicit which chain is being selected and what inputs will be passed along, which is useful for debugging and maintenance.</li>
</ul>


<p>This pattern is often recommended in LangChain examples and documentation for multi-step reasoning tasks.</p>

### Define LLM

In [2]:
from langchain_community.llms import Ollama

# Specify the remote server's URL
llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")

/tmp/ipykernel_113603/4093896791.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi4:latest", base_url="http://127.0.0.1:11434")


### Input Libraries

In [3]:
import json
import re
from langchain.schema import BaseOutputParser
from langchain.chains.router.llm_router import LLMRouterChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate



### Define Output Parser for Router_Prompt

In [3]:
# Custom Output Parser Definition

def clean_keys(d: dict) -> dict:
    new_d = {}
    for key, value in d.items():
        new_key = key.strip()
        if new_key.startswith('"') and new_key.endswith('"'):
            new_key = new_key[1:-1]
        if isinstance(value, dict):
            new_d[new_key] = clean_keys(value)
        else:
            new_d[new_key] = value
    return new_d

class CustomRouterOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # Extract the first JSON object from the text
        match = re.search(r"(\{.*\})", text, re.DOTALL)
        if not match:
            raise ValueError("No JSON object found in text.")
        json_str = match.group(1)
        try:
            result = json.loads(json_str)
        except json.JSONDecodeError as e:
            raise ValueError(f"Error decoding JSON: {json_str}") from e
        # Clean the keys to remove any extra quotes
        result = clean_keys(result)
        # Validate required keys
        if "destination" not in result:
            raise ValueError("Output JSON is missing the 'destination' key.")
        if "next_inputs" not in result:
            raise ValueError("Output JSON is missing the 'next_inputs' key.")
        if not isinstance(result["next_inputs"], dict):
            raise ValueError("The 'next_inputs' key must be a dictionary.")
        if "question" not in result["next_inputs"]:
            raise ValueError("The 'next_inputs' dictionary must include the 'question' key.")
        return result

    @property
    def _type(self) -> str:
        return "custom_router_output_parser"

# Instantiate the custom output parser
router_output_parser = CustomRouterOutputParser()

### Routing Logic and Formula Chain

In [4]:

# Router Prompt: Including new formulas

router_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "You are a routing assistant. Given the following question, decide which formula chain to use and return a valid JSON object with exactly two keys:\n"
        "1. destination: one of 'score_calculation', 'interest_calculation', or 'loan_calculation'.\n"
        "   - Use 'score_calculation' for questions about calculating customer score. \n"
        "   - Use 'interest_calculation' for questions about calculating deposit interest. \n"
        "   - Use 'loan_calculation' for questions about calculating remaining money for loans.\n"
        "2. next_inputs: a dictionary that includes the key 'question' mapping to the original question text.\n"
        "For example, if the question is 'How do I calculate the customer score?', return:\n"
        "{{\"destination\": \"score_calculation\", \"next_inputs\": {{\"question\": \"How do I calculate the customer score?\"}}}}\n"
        "Now, answer for the question: {question}"
    ),
    output_parser=router_output_parser
)


### Formula Chain --> Core of Calculations

In [18]:

# Create Formula Chains for Each Task/Formula


# Formula 1: Customer Score Calculation
score_calculation_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Customer Score Calculation:\n"
        "For the question: {question}, calculate the customer's score using the formula:\n"
        "    score = log((years_in_bank)^3) / age\n"
        "where 'years_in_bank' is the number of years the customer has had an account in our bank, and 'age' is the customer's age.\n"
        
    )
)
score_calculation_chain = LLMChain(llm=llm, prompt=score_calculation_prompt, verbose=True)

# Formula 2: Deposit Interest Calculation
interest_calculation_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Deposit Interest Calculation:\n"
        "For the question: {question}, calculate the deposit interest using the formula:\n"
        "    interest = ((deposit_amount * months) / 12) * 0.25\n"
        "where 'deposit_amount' is the amount of money deposited and 'months' is the number of months the money has been deposited.\n"
        "Provide only the arithmetic expression that calculates the interest.\n"
        "For example, if deposit_amount=2000000 and months=9, output something like: ((2000000 * 9)/12)*0.25"
    )
)
interest_calculation_chain = LLMChain(llm=llm, prompt=interest_calculation_prompt, verbose=True)

# Formula 3: Loan Calculation Chain (unchanged)
loan_calculation_prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "Loan Calculation Expert:\n"
        "For the question: {question}, use the formula:\n"
        "    amount_to_pay = total_amount - (number_of_paid_loans * amount_per_loan)\n"
        "to calculate the amount of money required to pay all the loans.\n"
        
    )
)
loan_calculation_chain = LLMChain(llm=llm, prompt=loan_calculation_prompt, verbose=True)


### Create Router Chain According to Router_Prompt and Destination_Chain

In [19]:
# Create the Router Chain with the Three Destination Chains

destination_chains = {
    "score_calculation": score_calculation_chain,
    "interest_calculation": interest_calculation_chain,
    "loan_calculation": loan_calculation_chain,
}

router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt,
    destination_chains=destination_chains
)

### Examples

In [7]:
# Example Usage: Testing Different Questions


# 1. Customer Score Calculation Question (Formula 1)
input_text1 = "How do I calculate the customer score?"
router_result1 = router_chain({"question": input_text1})
print("Router output for customer score:", router_result1)
destination1 = router_result1["destination"]
next_inputs1 = router_result1["next_inputs"]
if destination1 in destination_chains:
    final_result1 = destination_chains[destination1](next_inputs1)
    print(f"Final result from '{destination1}':", final_result1)
else:
    print("Destination not found:", destination1)


/tmp/ipykernel_7454/3651798248.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  router_result1 = router_chain({"question": input_text1})


Router output for customer score: {'question': 'How do I calculate the customer score?', 'destination': 'score_calculation', 'next_inputs': {'question': 'How do I calculate the customer score?'}}


> Entering new LLMChain chain...
Prompt after formatting:
Customer Score Calculation:
For the question: How do I calculate the customer score?, calculate the customer's score using the formula:
    score = log((years_in_bank)^3) / age
where 'years_in_bank' is the number of years the customer has had an account in our bank, and 'age' is the customer's age.
Explain the calculation step by step.

> Finished chain.
Final result from 'score_calculation': {'question': 'How do I calculate the customer score?', 'text': "To calculate the customer score using the provided formula:\n\n\\[ \\text{score} = \\frac{\\log((\\text{years\\_in\\_bank})^3)}{\\text{age}} \\]\n\nwe will go through each step of the calculation process.\n\n### Step 1: Identify Variables\n\n- **Years in Bank**: This is the number of

In [8]:
# 2. Deposit Interest Calculation Question (Formula 2)
# input_text2 = "How do I calculate the deposit interest?"
input_text2 = "من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره"
router_result2 = router_chain({"question": input_text2})
print("Router output for deposit interest:", router_result2)
destination2 = router_result2["destination"]
print(f"destination2 {destination2}")
next_inputs2 = router_result2["next_inputs"]
print(f"next_input2 {next_inputs2}")
if destination2 in destination_chains:
    final_result2 = destination_chains[destination2](next_inputs2)
    print(f"destination_chain2 = {destination_chains[destination2]}")
    print(f"Final result from '{destination2}':", final_result2)
else:
    print("Destination not found:", destination2)

/tmp/ipykernel_7293/1000696705.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  router_result2 = router_chain({"question": input_text2})


Router output for deposit interest: {'question': 'من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره', 'destination': 'interest_calculation', 'next_inputs': {'question': 'من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره'}}
destination2 interest_calculation
next_input2 {'question': 'من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره'}


> Entering new LLMChain chain...
Prompt after formatting:
Deposit Interest Calculation:
For the question: من میخوام ببینم سود سپرده بانک اگه ۵۰۰ میلیون پول بخوابونم چقدره, calculate the deposit interest using the formula:
    interest = ((deposit_amount * months) / 12) * 0.25
where 'deposit_amount' is the amount of money deposited and 'months' is the number of months the money has been deposited.
Explain the calculation step by step.

> Finished chain.
destination_chain2 = verbose=True prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template="Deposit Interest Calculatio

In [7]:
# 3. Loan Calculation Question (Loan Calculation)
# input_text3 = "How much money should I pay to pay all my loans to the bank?"
input_text3 = "How was the weather testerday?"
router_result3 = router_chain({"question": input_text3})
print("Router output for loan calculation:", router_result3)
destination3 = router_result3["destination"]
next_inputs3 = router_result3["next_inputs"]
if destination3 in destination_chains:
    final_result3 = destination_chains[destination3](next_inputs3)
    print(f"Final result from '{destination3}':", final_result3)
else:
    print("Destination not found:", destination3)

/tmp/ipykernel_395243/1638354673.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  router_result3 = router_chain({"question": input_text3})


Router output for loan calculation: {'question': 'How was the weather testerday?', 'destination': None, 'next_inputs': {'question': 'How was the weather yesterday?'}}
Destination not found: None


### Try to use python codes instead of mathchain to solve mathematics part

#### Consider to get missing parameters

In [ ]:
import re

# Define the formula as a string.
formula_str = "((deposit_amount * months) / 12) * 0.25"

def extract_interest_parameters(question: str) -> dict:
    """
    Extract numeric values from the question for interest calculation.
    This function uses regex to find all numbers in the text.
    
    For example:
      "من میخوام ببینم سود سپرده بانک اگه 500000000 پول بخوابونم چقدره"
    It will extract numbers and assume that the first number is deposit_amount
    and, if present, the second number is months.
    
    Note: In a real-world application, you might want to handle 
    non-Latin digits or multipliers like 'million' explicitly.
    """
    extracted = {}
    # Find all numeric occurrences (this will catch integers or decimals)
    numbers = re.findall(r"([\d,\.]+)", question)
    # Remove commas and convert to float
    numbers = [num.replace(',', '') for num in numbers]
    
    if numbers:
        # Assume the first number is deposit_amount.
        extracted['deposit_amount'] = float(numbers[0])
    if len(numbers) > 1:
        # If a second number exists, assume it's the months.
        extracted['months'] = float(numbers[1])
    
    return extracted

def get_missing_parameters(required_params: list, provided_params: dict) -> list:
    """Return a list of parameters that are missing."""
    return [param for param in required_params if param not in provided_params]

# Assume that the router has determined the user wants an interest calculation.
# The required parameters for this formula:
required_params = ['deposit_amount', 'months']

## 


In [2]:
# Example user question (in this case partly in Persian):
user_question = "من میخوام ببینم سود سپرده بانک اگه 500000000 پول بخوابونم چقدره"

# Step 1: Extract any provided parameters.
provided_params = extract_interest_parameters(user_question)
print("Extracted parameters:", provided_params)

# Step 2: Identify missing parameters.
missing_params = get_missing_parameters(required_params, provided_params)
if missing_params:
    for param in missing_params:
        # In a real chat system, you would ask the user via the chat interface.
        # Here we use input() for demonstration.
        value_str = input(f"Please enter a value for {param}: ")
        try:
            provided_params[param] = float(value_str)
        except ValueError:
            raise ValueError(f"Invalid value provided for {param}.")

# Step 3: Evaluate the formula using eval().
try:
    result_value = eval(formula_str, provided_params)
except Exception as e:
    raise RuntimeError(f"Error evaluating the formula: {e}")

# Step 4: Round the result to 2 decimal places (if needed).
if isinstance(result_value, float) and result_value != int(result_value):
    result_value = round(result_value, 2)
else:
    result_value = int(result_value)

print(f"Answer: {result_value}")

Extracted parameters: {'deposit_amount': 500000000.0}
Answer: 52083333.33


## Get missing params from user

In [20]:
# Define the required parameters for each formula
required_params_map = {
    "score_calculation": ["years_in_bank", "age"],
    "interest_calculation": ["deposit_amount", "months"],
    "loan_calculation": ["total_amount", "number_of_paid_loans", "amount_per_loan"]
}

In [21]:
# Helper function: Check for missing parameters and ask the user to supply them.
def fill_missing_parameters(next_inputs, required_params):
    """
    Checks if each required parameter is present in next_inputs.
    If not, prompts the user to input the missing value.
    Returns the updated next_inputs dictionary.
    """
    for param in required_params:
        # Check if the parameter is missing or empty.
        if param not in next_inputs or not next_inputs[param]:
            value = input(f"Please provide a value for '{param}': ")
            next_inputs[param] = value
    return next_inputs

In [26]:


# -------------------------------
# Example: Customer Score Calculation Question (Formula 1)
# -------------------------------
# input_text1 = "How do I calculate the customer score?"
input_text1 = "من میخوام ببینم سود سپرده بانک اگه مبلغی پول بخوابونم چقدره"
router_result1 = router_chain({"question": input_text1})
print("Router output for customer score:", router_result1)

destination1 = router_result1["destination"]
next_inputs1 = router_result1["next_inputs"]

# Check if there are any missing parameters for the selected destination
if destination1 in required_params_map:
    missing_params = [p for p in required_params_map[destination1] if p not in next_inputs1 or not next_inputs1[p]]
    if missing_params:
        print("Missing parameters for", destination1, ":", missing_params)
        # Ask the user to input missing parameters.
        next_inputs1 = fill_missing_parameters(next_inputs1, required_params_map[destination1])

# Now call the selected formula chain with the updated inputs
if destination1 in destination_chains:
    final_result1 = destination_chains[destination1](next_inputs1)
    print(f"Final result from '{destination1}':", final_result1)
else:
    print("Destination not found:", destination1)


Router output for customer score: {'question': 'من میخوام ببینم سود سپرده بانک اگه مبلغی پول بخوابونم چقدره', 'destination': 'interest_calculation', 'next_inputs': {'question': 'من میخوام ببینم سود سپرده بانک اگه مبلغی پول بخوابونم چقدره'}}
Missing parameters for interest_calculation : ['deposit_amount', 'months']


> Entering new LLMChain chain...
Prompt after formatting:
Deposit Interest Calculation:
For the question: من میخوام ببینم سود سپرده بانک اگه مبلغی پول بخوابونم چقدره, calculate the deposit interest using the formula:
    interest = ((deposit_amount * months) / 12) * 0.25
where 'deposit_amount' is the amount of money deposited and 'months' is the number of months the money has been deposited.
Provide only the arithmetic expression that calculates the interest.
For example, if deposit_amount=2000000 and months=9, output something like: ((2000000 * 9)/12)*0.25

> Finished chain.
Final result from 'interest_calculation': {'question': 'من میخوام ببینم سود سپرده بانک اگه مبلغی پو

In [4]:
# from langchain.chains import LLMMathChain
# llm_math = LLMMathChain.from_llm(llm, verbose=True)
# llm_math.prompt.template
test_expression = "((15000 * 4) / 12) * 0.25"
computed_result = math_chain.invoke(test_expression)
print("Computed numeric result:", computed_result)




> Entering new LLMMathChain chain...
((15000 * 4) / 12) * 0.25Answer: 125.00
> Finished chain.
Computed numeric result: {'question': '((15000 * 4) / 12) * 0.25', 'answer': 'Answer: 125.00'}


In [23]:
# Helper function to decide if a value is missing or a placeholder
def is_missing_value(value):
    if value is None:
        return True
    if isinstance(value, str):
        stripped = value.strip()
        # Consider empty strings or common placeholder words as missing
        if stripped == "" or stripped.lower() in ["default", "n/a", "none"]:
            return True
    return False

In [24]:
# Helper function to prompt for missing parameters
def fill_missing_parameters(next_inputs, required_params):
    """
    For each required parameter, if it's missing or invalid,
    prompt the user to provide a value and update the inputs.
    """
    for param in required_params:
        if param not in next_inputs or is_missing_value(next_inputs[param]):
            value = input(f"Please provide a value for '{param}': ")
            next_inputs[param] = value
    return next_inputs

In [25]:
# For example, suppose we have a question for deposit interest calculation.
input_text = "Calculate deposit interest for my deposit."

# Use the router chain to get the initial routing result.
router_result = router_chain({"question": input_text})
print("Router output:", router_result)

# Extract the destination key and next_inputs dictionary.
destination = router_result["destination"]
next_inputs = router_result["next_inputs"]

# Check if there are missing parameters for the selected formula.
if destination in required_params_map:
    missing_params = [p for p in required_params_map[destination] if p not in next_inputs or is_missing_value(next_inputs[p])]
    if missing_params:
        print("Missing parameters for", destination, ":", missing_params)
        next_inputs = fill_missing_parameters(next_inputs, required_params_map[destination])

# Now, execute the selected formula chain with the complete inputs.
if destination in destination_chains:
    final_result = destination_chains[destination](next_inputs)
    print(f"Final result from '{destination}':", final_result)
else:
    print("Destination not found:", destination)

Router output: {'question': 'Calculate deposit interest for my deposit.', 'destination': 'interest_calculation', 'next_inputs': {'question': 'Calculate deposit interest for my deposit.'}}
Missing parameters for interest_calculation : ['deposit_amount', 'months']


> Entering new LLMChain chain...
Prompt after formatting:
Deposit Interest Calculation:
For the question: Calculate deposit interest for my deposit., calculate the deposit interest using the formula:
    interest = ((deposit_amount * months) / 12) * 0.25
where 'deposit_amount' is the amount of money deposited and 'months' is the number of months the money has been deposited.
Provide only the arithmetic expression that calculates the interest.
For example, if deposit_amount=2000000 and months=9, output something like: ((2000000 * 9)/12)*0.25

> Finished chain.
Final result from 'interest_calculation': {'question': 'Calculate deposit interest for my deposit.', 'deposit_amount': '4000', 'months': '3', 'text': 'To calculate the d